# Sanity checks

In [2]:
# Imports
import sys

sys.path.append("../..")  # So we can import our modules

import os
import pandas as pd

## Frame extraction

### Pornography-800

In [3]:
paths = [
    "/nas-ctm01/datasets/public/BIOMETRICS/pornography-database/data/Database/vNonPornEasy",
    "/nas-ctm01/datasets/public/BIOMETRICS/pornography-database/data/Database/vNonPornDifficulty",
    "/nas-ctm01/datasets/public/BIOMETRICS/pornography-database/data/Database/vPorn",
]

#### even-20

In [31]:
# check if data file is available
data_loc_800_even_20 = "/nas-ctm01/datasets/public/BIOMETRICS/pornography-database/data-processed/even-20"
os.path.isfile(f"{data_loc_800_even_20}/data.csv")

True

In [32]:
df_800_even_20 = pd.read_csv(f"{data_loc_800_even_20}/data.csv")
df_800_even_20["video"] = [frame_name.split("#")[0] for frame_name in df_800_even_20["frame"]]
df_800_even_20

,frame,label,video
0,vNonPorn152#0.jpg,0,vNonPorn152
1,vNonPorn152#1.jpg,0,vNonPorn152
2,vNonPorn152#2.jpg,0,vNonPorn152
3,vNonPorn152#3.jpg,0,vNonPorn152
4,vNonPorn152#4.jpg,0,vNonPorn152
...,...,...,...
15955,vPorn346#15.jpg,1,vPorn346
15956,vPorn346#16.jpg,1,vPorn346
15957,vPorn346#17.jpg,1,vPorn346
15958,vPorn346#18.jpg,1,vPorn346


In [33]:
agg = {"video": "first", "label": "first", "frame": "size"}
df_800_even_20_videos = df_800_even_20[["video", "frame", "label"]]
df_800_even_20_videos = df_800_even_20_videos.groupby("video").aggregate(agg).reset_index(drop=True)
df_800_even_20_videos = df_800_even_20_videos.rename(columns={"frame": "frame_count"})
df_800_even_20_videos

,video,label,frame_count
0,vNonPorn1,0,20
1,vNonPorn10,0,20
2,vNonPorn100,0,20
3,vNonPorn1006,0,20
4,vNonPorn1007,0,20
...,...,...,...
793,vPorn94,1,20
794,vPorn95,1,20
795,vPorn96,1,20
796,vPorn97,1,20


In [34]:
for path in paths:
    videos_in_path = 0
    for v in os.listdir(path):
        if v.startswith("."):
            continue
        
        videos_in_path += 1
        v_name = os.path.splitext(v)[0]
        
        if len(df_800_even_20_videos[df_800_even_20_videos["video"] == v_name]) == 0:
            print(f"{v_name} not in dataset (it belongs to {path})")
        else:
            df_800_even_20_videos.loc[df_800_even_20_videos["video"] == v_name, "folder"] = os.path.basename(path)
    
    print(f"videos in {path}: {videos_in_path}")

videos in /nas-ctm01/datasets/public/BIOMETRICS/pornography-database/data/Database/vNonPornEasy: 200
videos in /nas-ctm01/datasets/public/BIOMETRICS/pornography-database/data/Database/vNonPornDifficulty: 198
videos in /nas-ctm01/datasets/public/BIOMETRICS/pornography-database/data/Database/vPorn: 400


In [35]:
df_800_even_20_videos

,video,label,frame_count,folder
0,vNonPorn1,0,20,vNonPornEasy
1,vNonPorn10,0,20,vNonPornDifficulty
2,vNonPorn100,0,20,vNonPornEasy
3,vNonPorn1006,0,20,vNonPornDifficulty
4,vNonPorn1007,0,20,vNonPornDifficulty
...,...,...,...,...
793,vPorn94,1,20,vPorn
794,vPorn95,1,20,vPorn
795,vPorn96,1,20,vPorn
796,vPorn97,1,20,vPorn


In [36]:
# videos from which it was not possible to extract 20 frames
df_800_even_20_videos[df_800_even_20_videos["frame_count"] < 20]

,video,label,frame_count,folder


In [37]:
# NOTE: I accidentally removed one video from its original location

print("number of videos:", len(df_800_even_20_videos))
print("number of non-porn frames:", len(df_800_even_20[df_800_even_20["label"] == 0]))
print("number of porn frames:", len(df_800_even_20[df_800_even_20["label"] == 1]))

number of videos: 798
number of non-porn frames: 7960
number of porn frames: 8000


#### middle-20

In [23]:
# check if data file is available
data_loc_800_middle_20 = "/nas-ctm01/datasets/public/BIOMETRICS/pornography-database/data-processed/middle-20"
os.path.isfile(f"{data_loc_800_middle_20}/data.csv")

True

In [24]:
df_800_middle_20 = pd.read_csv(f"{data_loc_800_middle_20}/data.csv")
df_800_middle_20["video"] = [frame_name.split("#")[0] for frame_name in df_800_middle_20["frame"]]
df_800_middle_20

,frame,label,video
0,vNonPorn152#0.jpg,0,vNonPorn152
1,vNonPorn152#1.jpg,0,vNonPorn152
2,vNonPorn152#2.jpg,0,vNonPorn152
3,vNonPorn152#3.jpg,0,vNonPorn152
4,vNonPorn152#4.jpg,0,vNonPorn152
...,...,...,...
15955,vPorn346#15.jpg,1,vPorn346
15956,vPorn346#16.jpg,1,vPorn346
15957,vPorn346#17.jpg,1,vPorn346
15958,vPorn346#18.jpg,1,vPorn346


In [25]:
agg = {"video": "first", "label": "first", "frame": "size"}
df_800_middle_20_videos = df_800_middle_20[["video", "frame", "label"]]
df_800_middle_20_videos = df_800_middle_20_videos.groupby("video").aggregate(agg).reset_index(drop=True)
df_800_middle_20_videos = df_800_middle_20_videos.rename(columns={"frame": "frame_count"})
df_800_middle_20_videos

,video,label,frame_count
0,vNonPorn1,0,20
1,vNonPorn10,0,20
2,vNonPorn100,0,20
3,vNonPorn1006,0,20
4,vNonPorn1007,0,20
...,...,...,...
793,vPorn94,1,20
794,vPorn95,1,20
795,vPorn96,1,20
796,vPorn97,1,20


In [26]:
for path in paths:
    videos_in_path = 0
    for v in os.listdir(path):
        if v.startswith("."):
            continue
        
        videos_in_path += 1
        v_name = os.path.splitext(v)[0]
        
        if len(df_800_middle_20_videos[df_800_middle_20_videos["video"] == v_name]) == 0:
            print(f"{v_name} not in dataset (it belongs to {path})")
        else:
            df_800_middle_20_videos.loc[df_800_middle_20_videos["video"] == v_name, "folder"] = os.path.basename(path)
    
    print(f"videos in {path}: {videos_in_path}")

videos in /nas-ctm01/datasets/public/BIOMETRICS/pornography-database/data/Database/vNonPornEasy: 200
videos in /nas-ctm01/datasets/public/BIOMETRICS/pornography-database/data/Database/vNonPornDifficulty: 198
videos in /nas-ctm01/datasets/public/BIOMETRICS/pornography-database/data/Database/vPorn: 400


In [27]:
df_800_middle_20_videos

,video,label,frame_count,folder
0,vNonPorn1,0,20,vNonPornEasy
1,vNonPorn10,0,20,vNonPornDifficulty
2,vNonPorn100,0,20,vNonPornEasy
3,vNonPorn1006,0,20,vNonPornDifficulty
4,vNonPorn1007,0,20,vNonPornDifficulty
...,...,...,...,...
793,vPorn94,1,20,vPorn
794,vPorn95,1,20,vPorn
795,vPorn96,1,20,vPorn
796,vPorn97,1,20,vPorn


In [28]:
# videos from which it was not possible to extract 20 frames
df_800_middle_20_videos[df_800_middle_20_videos["frame_count"] < 20]

,video,label,frame_count,folder


In [29]:
# NOTE: I accidentally removed one video from its original location

print("number of videos:", len(df_800_middle_20_videos))
print("number of non-porn frames:", len(df_800_middle_20[df_800_middle_20["label"] == 0]))
print("number of porn frames:", len(df_800_middle_20[df_800_middle_20["label"] == 1]))

number of videos: 798
number of non-porn frames: 7960
number of porn frames: 8000


#### even-40

In [38]:
# check if data file is available
data_loc_800_even_40 = "/nas-ctm01/datasets/public/BIOMETRICS/pornography-database/data-processed/even-40"
os.path.isfile(f"{data_loc_800_even_40}/data.csv")

True

In [39]:
df_800_even_40 = pd.read_csv(f"{data_loc_800_even_40}/data.csv")
df_800_even_40["video"] = [frame_name.split("#")[0] for frame_name in df_800_even_40["frame"]]
df_800_even_40

,frame,label,video
0,vNonPorn152#0.jpg,0,vNonPorn152
1,vNonPorn152#1.jpg,0,vNonPorn152
2,vNonPorn152#2.jpg,0,vNonPorn152
3,vNonPorn152#3.jpg,0,vNonPorn152
4,vNonPorn152#4.jpg,0,vNonPorn152
...,...,...,...
31915,vPorn346#35.jpg,1,vPorn346
31916,vPorn346#36.jpg,1,vPorn346
31917,vPorn346#37.jpg,1,vPorn346
31918,vPorn346#38.jpg,1,vPorn346


In [40]:
agg = {"video": "first", "label": "first", "frame": "size"}
df_800_even_40_videos = df_800_even_40[["video", "frame", "label"]]
df_800_even_40_videos = df_800_even_40_videos.groupby("video").aggregate(agg).reset_index(drop=True)
df_800_even_40_videos = df_800_even_40_videos.rename(columns={"frame": "frame_count"})
df_800_even_40_videos

,video,label,frame_count
0,vNonPorn1,0,40
1,vNonPorn10,0,40
2,vNonPorn100,0,40
3,vNonPorn1006,0,40
4,vNonPorn1007,0,40
...,...,...,...
793,vPorn94,1,40
794,vPorn95,1,40
795,vPorn96,1,40
796,vPorn97,1,40


In [41]:
for path in paths:
    videos_in_path = 0
    for v in os.listdir(path):
        if v.startswith("."):
            continue
        
        videos_in_path += 1
        v_name = os.path.splitext(v)[0]
        
        if len(df_800_even_40_videos[df_800_even_40_videos["video"] == v_name]) == 0:
            print(f"{v_name} not in dataset (it belongs to {path})")
        else:
            df_800_even_40_videos.loc[df_800_even_40_videos["video"] == v_name, "folder"] = os.path.basename(path)
    
    print(f"videos in {path}: {videos_in_path}")

videos in /nas-ctm01/datasets/public/BIOMETRICS/pornography-database/data/Database/vNonPornEasy: 200
videos in /nas-ctm01/datasets/public/BIOMETRICS/pornography-database/data/Database/vNonPornDifficulty: 198
videos in /nas-ctm01/datasets/public/BIOMETRICS/pornography-database/data/Database/vPorn: 400


In [42]:
# videos from which it was not possible to extract 40 frames
df_800_even_40_videos[df_800_even_40_videos["frame_count"] < 40]

,video,label,frame_count,folder


In [43]:
# NOTE: I accidentally removed one video from its original location

print("number of videos:", len(df_800_even_40_videos))
print("number of non-porn frames:", len(df_800_even_40[df_800_even_40["label"] == 0]))
print("number of porn frames:", len(df_800_even_40[df_800_even_40["label"] == 1]))

number of videos: 798
number of non-porn frames: 15920
number of porn frames: 16000


### Pornography-2k

#### middle-20

In [44]:
# check if data file is available
data_loc_2k_middle_20 = "/nas-ctm01/datasets/public/BIOMETRICS/pornography-2k-db/data-processed/middle-20"
os.path.isfile(f"{data_loc_2k_middle_20}/data.csv")

True

In [45]:
df_2k_middle_20 = pd.read_csv(f"{data_loc_2k_middle_20}/data.csv")
df_2k_middle_20["video"] = [frame_name.split("#")[0] for frame_name in df_2k_middle_20["frame"]]
df_2k_middle_20

,frame,label,video
0,vPorn000842#0.jpg,1,vPorn000842
1,vPorn000842#1.jpg,1,vPorn000842
2,vPorn000842#2.jpg,1,vPorn000842
3,vPorn000842#3.jpg,1,vPorn000842
4,vPorn000842#4.jpg,1,vPorn000842
...,...,...,...
39995,vPorn000431#15.jpg,1,vPorn000431
39996,vPorn000431#16.jpg,1,vPorn000431
39997,vPorn000431#17.jpg,1,vPorn000431
39998,vPorn000431#18.jpg,1,vPorn000431


In [46]:
agg = {"video": "first", "label": "first", "frame": "size"}
df_2k_middle_20_videos = df_2k_middle_20[["video", "frame", "label"]]
df_2k_middle_20_videos = df_2k_middle_20_videos.groupby("video").aggregate(agg).reset_index(drop=True)
df_2k_middle_20_videos = df_2k_middle_20_videos.rename(columns={"frame": "frame_count"})
df_2k_middle_20_videos

,video,label,frame_count
0,vNonPorn000001,0,20
1,vNonPorn000002,0,20
2,vNonPorn000003,0,20
3,vNonPorn000004,0,20
4,vNonPorn000005,0,20
...,...,...,...
1995,vPorn000996,1,20
1996,vPorn000997,1,20
1997,vPorn000998,1,20
1998,vPorn000999,1,20


In [47]:
# videos from which it was not possible to extract 20 frames
df_2k_middle_20_videos[df_2k_middle_20_videos["frame_count"] < 20]

,video,label,frame_count


In [48]:
print("number of videos:", len(df_2k_middle_20_videos))
print("number of non-porn frames:", len(df_2k_middle_20[df_2k_middle_20["label"] == 0]))
print("number of porn frames:", len(df_2k_middle_20[df_2k_middle_20["label"] == 1]))

number of videos: 2000
number of non-porn frames: 20000
number of porn frames: 20000


#### even-20

In [49]:
# check if data file is available
data_loc_2k_even_20 = "/nas-ctm01/datasets/public/BIOMETRICS/pornography-2k-db/data-processed/even-20"
os.path.isfile(f"{data_loc_2k_even_20}/data.csv")

True

In [50]:
df_2k_even_20 = pd.read_csv(f"{data_loc_2k_even_20}/data.csv")
df_2k_even_20["video"] = [frame_name.split("#")[0] for frame_name in df_2k_even_20["frame"]]
df_2k_even_20

,frame,label,video
0,vPorn000842#0.jpg,1,vPorn000842
1,vPorn000842#1.jpg,1,vPorn000842
2,vPorn000842#2.jpg,1,vPorn000842
3,vPorn000842#3.jpg,1,vPorn000842
4,vPorn000842#4.jpg,1,vPorn000842
...,...,...,...
39995,vPorn000431#15.jpg,1,vPorn000431
39996,vPorn000431#16.jpg,1,vPorn000431
39997,vPorn000431#17.jpg,1,vPorn000431
39998,vPorn000431#18.jpg,1,vPorn000431


In [51]:
agg = {"video": "first", "label": "first", "frame": "size"}
df_2k_even_20_videos = df_2k_even_20[["video", "frame", "label"]]
df_2k_even_20_videos = df_2k_even_20_videos.groupby("video").aggregate(agg).reset_index(drop=True)
df_2k_even_20_videos = df_2k_even_20_videos.rename(columns={"frame": "frame_count"})
df_2k_even_20_videos

,video,label,frame_count
0,vNonPorn000001,0,20
1,vNonPorn000002,0,20
2,vNonPorn000003,0,20
3,vNonPorn000004,0,20
4,vNonPorn000005,0,20
...,...,...,...
1995,vPorn000996,1,20
1996,vPorn000997,1,20
1997,vPorn000998,1,20
1998,vPorn000999,1,20


In [52]:
# videos from which it was not possible to extract 20 frames
df_2k_even_20_videos[df_2k_even_20_videos["frame_count"] < 20]

,video,label,frame_count


In [53]:
print("number of videos:", len(df_2k_even_20_videos))
print("number of non-porn frames:", len(df_2k_even_20[df_2k_even_20["label"] == 0]))
print("number of porn frames:", len(df_2k_even_20[df_2k_even_20["label"] == 1]))

number of videos: 2000
number of non-porn frames: 20000
number of porn frames: 20000


## Model training and testing

In [ ]:
model_path = "../../results/pornography-2k/cnns/data-aug/even-20/models/vgg19_freeze_False_epochs_50_batch_16_optim_sgd_aug_True_split_10_20.pth"
results_path = "../../results/pornography-2k/cnns/data-aug/even-20/results/vgg19_freeze_False_epochs_50_batch_16_optim_sgd_aug_True_split_10_20.csv"

In [ ]:
df_results = pd.read_csv(results_path)
df_results

In [ ]:
print(
    "Frames correctly classified as pornographic:",
    len(df_results[(df_results["Target"] == 1) & (df_results["Prediction"] == 1)]),
)
print(
    "Frames incorrectly classified as pornographic",
    len(df_results[(df_results["Target"] == 0) & (df_results["Prediction"] == 1)]),
)
print(
    "Frames correctly classified as non-pornographic",
    len(df_results[(df_results["Target"] == 0) & (df_results["Prediction"] == 0)]),
)
print(
    "Frames incorrectly classified as pornographic",
    len(df_results[(df_results["Target"] == 1) & (df_results["Prediction"] == 0)]),
)

In [ ]:
df_results["Video"] = df_results["Frame"].str.split("#").str[0]
df_results

In [ ]:
def separate_frames(row):
    target = row["Target"]

    correct_frames, incorrect_frames, correct_confidences, incorrect_confidences = (
        [],
        [],
        [],
        [],
    )

    for frame, prediction, confidence in zip(
        row["Frame"], row["Prediction"], row["Confidence"]
    ):
        if target == prediction:  # Correct prediction
            correct_frames.append(frame)
            correct_confidences.append(confidence)
        else:  # Incorrect prediction
            incorrect_frames.append(frame)
            incorrect_confidences.append(confidence)

    return pd.Series(
        [correct_frames, incorrect_frames, correct_confidences, incorrect_confidences],
        index=[
            "Correct_frames",
            "Incorrect_frames",
            "Correct_confidences",
            "Incorrect_confidences",
        ],
    )


# Group by video and aggregate columns
grouped_df = (
    df_results.groupby("Video")
    .agg(
        {
            "Target": "first",
            "Prediction": lambda x: list(x),
            "Frame": lambda x: list(x),
            "Confidence": lambda x: list(x),
        }
    )
    .reset_index()
)

# Apply aggregation function
grouped_df[
    [
        "Correct_frames",
        "Incorrect_frames",
        "Correct_confidences",
        "Incorrect_confidences",
    ]
] = grouped_df.apply(separate_frames, axis=1)

# Drop unnecessary columns
grouped_df.drop(["Frame", "Prediction", "Confidence"], axis=1, inplace=True)

grouped_df

In [ ]:
# Videos with both correctly and incorrectly classified frames
grouped_df[
    grouped_df.apply(
        lambda row: len(row["Correct_frames"]) > 0 and len(row["Incorrect_frames"]) > 0,
        axis=1,
    )
]

In [ ]:
# Videos with only incorrectly classified frames
grouped_df[
    grouped_df.apply(
        lambda row: len(row["Correct_frames"]) == 0
        and len(row["Incorrect_frames"]) > 0,
        axis=1,
    )
]

In [ ]:
# Videos with only correctly classified frames
grouped_df[grouped_df.apply(lambda row: len(row["Incorrect_frames"]) == 0, axis=1)]

In [ ]:
grouped_df[grouped_df["Video"] == "vPorn000028"]

In [ ]:
# Videos which would have been correctly classified if in a majority voting scheme
print(
    f"Videos with more correctly than incorrectly classified frames: {len(grouped_df[grouped_df.apply(lambda row: len(row['Correct_frames']) > len(row['Incorrect_frames']), axis=1)])} / {len(grouped_df)}"
)

In [ ]:
df_results[df_results["Frame"] == "vPorn000885#17.jpg"]

- vPorn000885#17 (5297) - really interesting example because when predicting porn, the model looks at the logo, but to predict non-porn the model looks at the sexual organ
- vPorn000676#9 (1789), vPorn000687#14 (3434) - interesting to see how the model is paying attention to the logo
- vNonPorn000367#3 (2383) - when it classifies a non-porn as porn, it is usually paying attention to actually relevant stuff
- vNonPorn000808#0 (4100) - here it was paying attention to the hands (skin)
- vNonPorn000966#0 (1520) - here it payed attention two the pigs, it might have been because of the shape or the color, which is similar to that of skin
- vPorn000005#0 (4140), vPorn000034#6 (686) - here, for example, although there is a logo, it payed attention to the skin areas